##LightGBM

A notebook demonstrating the use of K-means correlation clustering (HRP was experimented with but resulting clusters were not intuitive) to select features that characterise most risks. In this demonstration, the model was trained on data sourced from AlphaVantage and saved to flat files. Unfortunately the original data is no longer available.

This is the training notebook for the trend component of ETF returns using LightGBM, several models have been used to forecast the noise component and I hope to load these in the near future.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install optuna

import pandas as pd
import os
import numpy as np
import lightgbm as lgb
import optuna
from statsmodels.tsa.seasonal import STL
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
start_date, end_date = '2006-10-01', '2016-11-30'

volume = pd.read_csv('/content/drive/My Drive/Data/ETFs/volume.csv', parse_dates=['Date'], dayfirst=True).set_index('Date').loc[start_date:end_date]
close = pd.read_csv('/content/drive/My Drive/Data/ETFs/close.csv', parse_dates=['Date'], dayfirst=True).set_index('Date').loc[start_date:end_date]

# '_s' appended to names denotes structured
volume_levels_s = volume.reindex(pd.date_range(start=start_date, end=end_date, freq='D')).ffill()
close_levels_s = close.reindex(pd.date_range(start=start_date, end=end_date, freq='D')).ffill()
close_ln_s = close_levels_s.applymap(lambda x: np.log(x) if pd.notnull(x) else x)
close_lnreturns_s = close_ln_s.diff()

In [ ]:
size = int(len(close_ln_s) * 0.7)

# in-sample dataframes
volume_levels, close_levels, close_ln, close_lnreturns = [df.iloc[:size] for df in [volume_levels_s, close_levels_s, close_ln_s, close_lnreturns_s]]

# out-of-sample dataframes (with '_oos' appended to names)
volume_levels_oos, close_levels_oos, close_ln_oos, close_lnreturns_oos = [df.iloc[size:] for df in [volume_levels_s, close_levels_s, close_ln_s, close_lnreturns_s]]


In [ ]:
number_of_assets = 500 # number of securities to narrow by volume

high_volume = volume_levels.iloc[-100:].mean().sort_values(ascending=False).head(number_of_assets).index.tolist()

close_lnuniverse = close_ln.loc[:, close_ln.columns.intersection(high_volume)]
close_lnreturnsuniverse = close_lnreturns.loc[:, close_lnreturns.columns.intersection(high_volume)]

close_lnreturnsuniverse.iloc[-5:]

,prf,pjp,ipff,iefa,sdog,ephe,urty,pbs,ugl,hys,...,mint,oef,ipo,iat,ryf,pst,xbi,tdtt,ixus,bsch
2013-11-07,-0.012659,-0.007750,-0.003751,-0.015864,-0.014615,-0.032012,-0.053835,-0.024271,-0.017029,-0.000380,...,0.000206,-0.011431,-0.027854,-0.009039,-0.010054,-0.005881,-0.018905,-0.002065,-0.012432,0.001347
2013-11-08,0.013937,0.037765,0.004539,0.005077,0.008104,0.001219,0.054905,0.022581,-0.031993,-0.001927,...,-0.000206,0.012677,0.011028,0.029312,0.016176,0.021285,0.040278,0.000826,0.000000,-0.004001
2013-11-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-11-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-11-11,0.001113,0.005041,0.002360,0.001699,0.001470,-0.011022,0.003839,0.000000,-0.003595,-0.001387,...,0.000515,0.000397,0.002532,-0.004639,0.000970,0.000679,0.012011,0.000413,0.002499,0.000839


In [ ]:
# absolute correlation
similarity = close_lnreturnsuniverse.corr().abs()

similarity = similarity.dropna(axis=0, how='all').dropna(axis=1, how='all')

scaler = StandardScaler()
scaled_data = scaler.fit_transform(similarity)

kmeans = KMeans(n_clusters=30, random_state=42)
clusters = kmeans.fit_predict(scaled_data)

clustered_assets = pd.DataFrame({'Asset': similarity.index, 'Cluster': clusters})

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
centroids = kmeans.cluster_centers_

closest_features = []
for i, centroid in enumerate(centroids):
    distances = np.linalg.norm(scaled_data[kmeans.labels_ == i] - centroid, axis=1)
    closest_index = np.argmin(distances)
    closest_feature = similarity.index[kmeans.labels_ == i][closest_index]
    closest_features.append(closest_feature)

log_prices = close_lnuniverse[closest_features]

log_prices

,fbt,mint,sdog,sivr,iwc,fxy,gsg,hys,bsch,tlt,...,xle,shv,gvi,fpf,rsp,flrn,pbj,eido,tfi,sjnk
2006-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-10-02,3.061333,NaN,NaN,NaN,3.876769,NaN,3.720378,NaN,NaN,4.304416,...,3.813748,NaN,NaN,NaN,3.689904,NaN,2.653312,NaN,NaN,NaN
2006-10-03,3.059458,NaN,NaN,NaN,3.876562,NaN,3.695855,NaN,NaN,4.303105,...,3.776318,NaN,NaN,NaN,3.687403,NaN,2.650068,NaN,NaN,NaN
2006-10-04,3.082598,NaN,NaN,NaN,3.894612,NaN,3.705491,NaN,NaN,4.310370,...,3.790917,NaN,NaN,NaN,3.700857,NaN,2.663680,NaN,NaN,NaN
2006-10-05,3.095351,NaN,NaN,NaN,3.909981,NaN,3.715034,NaN,NaN,4.303335,...,3.809902,NaN,NaN,NaN,3.706253,NaN,2.669448,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-11-07,4.108790,4.574824,3.367399,3.060115,4.176493,4.602467,3.428813,4.462754,3.069912,4.561709,...,4.339654,4.693364,4.636572,2.688120,4.151921,3.387977,3.192737,3.153420,3.728509,3.216673
2013-11-08,4.145830,4.574618,3.375503,3.054944,4.196149,4.590868,3.434954,4.460826,3.065911,4.537566,...,4.354450,4.693364,4.632785,2.668755,4.164741,3.387301,3.202421,3.163913,3.725356,3.216032
2013-11-09,4.145830,4.574618,3.375503,3.054944,4.196149,4.590868,3.434954,4.460826,3.065911,4.537566,...,4.354450,4.693364,4.632785,2.668755,4.164741,3.387301,3.202421,3.163913,3.725356,3.216032
2013-11-10,4.145830,4.574618,3.375503,3.054944,4.196149,4.590868,3.434954,4.460826,3.065911,4.537566,...,4.354450,4.693364,4.632785,2.668755,4.164741,3.387301,3.202421,3.163913,3.725356,3.216032


In [ ]:
def apply_stl_to_column(series, period=7): # weekly seasonality
    if series.dropna().empty:
        return None, None
    stl = STL(series.dropna(), period=period, robust=True)  # drop leading NaNs
    result = stl.fit()
    combined_seasonal_residual = result.seasonal + result.resid  # combine seasonal and residual
    return result.trend, combined_seasonal_residual

trend_df = pd.DataFrame(index=log_prices.index)
combined_df = pd.DataFrame(index=log_prices.index)

# parallelisation
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(apply_stl_to_column, log_prices[col], 21): col for col in log_prices.columns}

    for future in as_completed(futures):
        col = futures[future]
        trend, combined_seasonal_residual = future.result()

        if trend is not None:
            trend_df[col] = pd.Series(trend, index=log_prices[col].dropna().index)
            combined_df[col] = pd.Series(combined_seasonal_residual, index=log_prices[col].dropna().index)

trend_df.iloc[-5:]

,sdog,mint,sivr,fbt,iwc,bsch,fxy,gsg,hys,dgp,...,fpf,xle,shv,gvi,flrn,pbj,rsp,sjnk,eido,tfi
2013-11-07,3.383675,4.574814,3.083438,4.164710,4.197830,3.069368,4.598322,3.438264,4.463039,3.421890,...,2.687065,4.354941,4.693392,4.636301,3.387827,3.210454,4.167552,3.216370,3.178419,3.730617
2013-11-08,3.384586,4.574865,3.083510,4.164675,4.197369,3.069549,4.598062,3.435989,4.463310,3.422010,...,2.686825,4.355204,4.693387,4.636416,3.387826,3.210736,4.168431,3.216643,3.176619,3.731096
2013-11-09,3.385483,4.574916,3.083562,4.164629,4.196886,3.069725,4.597800,3.433705,4.463576,3.422078,...,2.686586,4.355450,4.693381,4.636528,3.387823,3.211002,4.169295,3.216914,3.174810,3.731571
2013-11-10,3.386365,4.574966,3.083594,4.164572,4.196380,3.069895,4.597536,3.431415,4.463837,3.422094,...,2.686347,4.355679,4.693376,4.636636,3.387819,3.211255,4.170145,3.217183,3.172993,3.732042
2013-11-11,3.387233,4.575016,3.083606,4.164503,4.195855,3.070061,4.597269,3.429119,4.464093,3.422058,...,2.686108,4.355892,4.693370,4.636740,3.387812,3.211494,4.170981,3.217449,3.171169,3.732509


Training and Validation. This block measures the average MAE across all observations h-steps ahead. The goal is to find params that are robust across asset classes.

In [ ]:
data_subset = trend_df

n_validation_windows = 3
n_forecast = 30

def objective(trial):
    # hyperparameters to tune
    num_leaves = trial.suggest_int('num_leaves', 40, 80)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 200, 400)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 20)
    feature_fraction = trial.suggest_float('feature_fraction', 0.5, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 0.9)
    bagging_freq = trial.suggest_int('bagging_freq', 2, 9)
    lambda_l1 = trial.suggest_float('lambda_l1', 1e-2, 3, log=True)
    lambda_l2 = trial.suggest_float('lambda_l2', 1e-5, 3, log=True)
    min_gain_to_split = trial.suggest_float('min_gain_to_split', 0.0, 0.5)

    aggregated_original_maes = []

    for column in data_subset.columns:
        series = data_subset[column].dropna()
        if len(series) < (n_validation_windows * n_forecast):
            continue

        n_validation = n_validation_windows * n_forecast
        train_series = series[:-n_validation]
        validation_series = series[-n_validation:]

        scaler = StandardScaler(with_mean=True, with_std=False)
        train_scaled = scaler.fit_transform(train_series.values.reshape(-1, 1)).flatten()

        train_x = np.arange(len(train_scaled)).reshape(-1, 1)
        train_y = train_scaled

        lgb_train = lgb.Dataset(train_x, train_y)

        # lightGBM parameters
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'num_leaves': num_leaves,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'min_child_samples': min_child_samples,
            'feature_fraction': feature_fraction,
            'bagging_fraction': bagging_fraction,
            'bagging_freq': bagging_freq,
            'lambda_l1': lambda_l1,
            'lambda_l2': lambda_l2,
            'min_gain_to_split': min_gain_to_split,
            'verbose': -1
        }

        # train the model
        gbm = lgb.train(params, lgb_train, num_boost_round=n_estimators)

        mae = walk_forward_forecast_and_retrain(gbm, scaler, n_forecast, series, n_validation_windows)

        aggregated_original_maes.append(mae)

    return np.mean(aggregated_original_maes)

def walk_forward_forecast_and_retrain(model, scaler, n_forecast, series, n_validation_windows):
    n_validation = n_validation_windows * n_forecast
    train_series = series[:-n_validation]
    validation_series = series[-n_validation:]

    mae_values = []

    for i in range(n_validation_windows):
        start_idx = i * n_forecast
        end_idx = start_idx + n_forecast
        current_validation = validation_series[start_idx:end_idx]

        # forecast for the next period
        validation_x = np.arange(len(train_series) + start_idx, len(train_series) + end_idx).reshape(-1, 1)
        predicted = model.predict(validation_x)
        predicted = scaler.inverse_transform(predicted.reshape(-1, 1)).flatten()

        # calculate MAE for the forecast and the actual values
        mae = mean_absolute_error(current_validation.values, predicted)
        mae_values.append(mae)

    return np.mean(mae_values)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

print(f'best trial: {study.best_trial.number}')
print(f'best MAE: {study.best_trial.value}')
print(f'best hyperparameters: {study.best_trial.params}')


[I 2024-03-01 19:14:48,372] A new study created in memory with name: no-name-c5f9ac8c-89dc-4e10-8f44-730c27c852be
[I 2024-03-01 19:14:49,316] Trial 0 finished with value: 0.07565764935059792 and parameters: {'num_leaves': 73, 'learning_rate': 0.11984163912951513, 'n_estimators': 302, 'max_depth': 2, 'min_child_samples': 8, 'feature_fraction': 0.6873834334466675, 'bagging_fraction': 0.5656143085464131, 'bagging_freq': 6, 'lambda_l1': 0.27191237130800383, 'lambda_l2': 3.318805847354643e-05, 'min_gain_to_split': 0.4658320214927204}. Best is trial 0 with value: 0.07565764935059792.
[I 2024-03-01 19:14:49,689] Trial 1 finished with value: 0.0517871645771966 and parameters: {'num_leaves': 44, 'learning_rate': 0.10107295912662707, 'n_estimators': 354, 'max_depth': 2, 'min_child_samples': 5, 'feature_fraction': 0.875057249656317, 'bagging_fraction': 0.8500070925821641, 'bagging_freq': 7, 'lambda_l1': 0.22957559110109169, 'lambda_l2': 0.023137155474268874, 'min_gain_to_split': 0.077266505304648

Best trial: 95
Best MAE: 0.028094592735252864
Best hyperparameters: {'num_leaves': 44, 'learning_rate': 0.24759859623131597, 'n_estimators': 385, 'max_depth': 6, 'min_child_samples': 5, 'feature_fraction': 0.7018745555372053, 'bagging_fraction': 0.6324069166306597, 'bagging_freq': 5, 'lambda_l1': 0.018319799581390715, 'lambda_l2': 0.02570786227424883, 'min_gain_to_split': 1.0389238477655346e-05}
